In [1]:
!pip install torch==2.3.0 torchaudio==2.3.0 torchvision==0.18.0
!pip install albumentations numpy pandas scikit_learn kaggle
!pip install resnest geffnet opencv-python pretrainedmodels tqdm Pillow packaging monai

Training the U-nets

In [2]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from monai.networks.nets import UNet
import torch.nn as nn
import torch.optim as optim

In [3]:
from google.colab import drive, files
from Kits2019 import Kits2019Dataset
from utils import get_transforms,prepare_datasets
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
print(sys.path)


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython']


In [5]:
data_dir = '/content/drive/MyDrive/filtered_data'
train_dataset, val_dataset, test_dataset, train_loader, val_loader, test_loader = prepare_datasets(data_dir)

for batch in train_loader:
    images, labels = batch
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break


torch.Size([1, 37, 512, 512])
torch.Size([1, 37, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


torch.Size([1, 35, 512, 512])
torch.Size([1, 35, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


torch.Size([1, 49, 512, 512])
torch.Size([1, 49, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


torch.Size([1, 32, 512, 512])
torch.Size([1, 32, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


Image batch shape: torch.Size([4, 1, 256, 256, 256])
Label batch shape: torch.Size([4, 1, 256, 256, 256])


## Training model

In [6]:
model_unet = UNet(
    spatial_dims=3,  # Correct value for 3D convolutions
    in_channels=1,   # Match input data channels
    out_channels=3,  # Number of segmentation classes
    channels=(16, 32, 64, 128, 256),  # Feature map sizes for each level
    strides=(2, 2, 2, 2),  # Downsampling factors for each level
)

In [7]:
def train_model_multiclass(
    model, train_loader, val_loader, device,
    num_epochs=15, lr=1e-4, patience=3, model_name="model_multiclass.pth"
):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Early stopping parameters
    best_loss = float('inf')
    counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for i, (images, masks) in enumerate(train_loader):
            print(f"\n[Train] Epoch {epoch}, Batch {i}")
            print(f"Input images shape: {images.shape}")  # No squeezing here
            images = images.to(device)  # Shape: [B, C_in, D, H, W]
            print(f"Images moved to {device}. Shape: {images.shape}")

            print(f"Input masks shape: {masks.shape}")
            masks = masks.to(device).long()  # Shape: [B, C_out, D, H, W]
            print(f"Masks moved to {device}. Shape: {masks.shape}")

            optimizer.zero_grad()
            outputs = model(images)  # Shape: [B, C_out, D, H, W]
            print(f"Model outputs shape: {outputs.shape}")

            # Ensure masks are compatible with the loss function
            loss = criterion(outputs, masks.squeeze(1))  # Remove channel dim for masks if necessary
            print(f"Calculated loss: {loss.item()}")

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if i % 10 == 0:
                print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")

        avg_train_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch} - Average Training Loss: {avg_train_loss}")

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for j, (images, masks) in enumerate(val_loader):
                print(f"\n[Validation] Epoch {epoch}, Batch {j}")
                print(f"Input images shape: {images.shape}")
                images = images.to(device)
                print(f"Images moved to {device}. Shape: {images.shape}")

                print(f"Input masks shape: {masks.shape}")
                masks = masks.to(device).long()
                print(f"Masks moved to {device}. Shape: {masks.shape}")

                outputs = model(images)
                print(f"Model outputs shape: {outputs.shape}")

                loss = criterion(outputs, masks.squeeze(1))  # Remove channel dim for masks if necessary
                print(f"Calculated validation loss: {loss.item()}")

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch} - Validation Loss: {avg_val_loss}")

        if avg_val_loss < best_loss - 1e-4:
            best_loss = avg_val_loss
            counter = 0
            print(f"Validation loss improved to {best_loss}. Saving model.")
            torch.save(model.state_dict(), model_name)
        else:
            counter += 1
            print(f"No improvement in validation loss. Patience counter: {counter}/{patience}")

        if counter >= patience:
            print("Early stopping triggered.")
            break

    print("Training complete.")
    files.download(model_name)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train UNet
train_model_multiclass(
    model=model_unet,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_epochs=10,
    lr=1e-4,
    patience=3,
    model_name="unet_multiclass.pth"
)


torch.Size([1, 263, 512, 512])
torch.Size([1, 263, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


torch.Size([1, 22, 512, 512])
torch.Size([1, 22, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")


torch.Size([1, 26, 512, 512])
torch.Size([1, 26, 512, 512])


...

torch.Size([1, 299, 512, 512])
torch.Size([1, 299, 512, 512])


/usr/local/lib/python3.10/dist-packages/monai/transforms/spatial/array.py:497: UserWarning: `data_array` is not of type MetaTensor, assuming affine to be identity.
  warnings.warn("`data_array` is not of type MetaTensor, assuming affine to be identity.")



[Validation] Epoch 9, Batch 10
Input images shape: torch.Size([2, 1, 256, 256, 256])
Images moved to cuda. Shape: torch.Size([2, 1, 256, 256, 256])
Input masks shape: torch.Size([2, 1, 256, 256, 256])
Masks moved to cuda. Shape: torch.Size([2, 1, 256, 256, 256])
Model outputs shape: torch.Size([2, 3, 256, 256, 256])
Calculated validation loss: 0.3396092653274536
Epoch 9 - Validation Loss: 0.33523862199349835
Validation loss improved to 0.33523862199349835. Saving model.
Training complete.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>